# Smart Grid Optimization - Energy Systems Analysis

## Tier 2 AWS Project: Grid Data Analysis with Lambda

This notebook analyzes smart grid data processed by AWS Lambda to monitor grid performance, renewable integration, and stability.

**What you'll learn:**
- Generate and upload grid sensor data
- Trigger Lambda processing
- Query results from DynamoDB
- Analyze load patterns and renewable integration
- Visualize grid stability metrics

---

## Setup: Import Libraries

In [ ]:
import json
import warnings
from datetime import datetime, timedelta
from decimal import Decimal
from pathlib import Path

import boto3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)
plt.rcParams["font.size"] = 10

print("✓ Libraries imported successfully")

## Configuration: AWS Setup

Update these values with your AWS resources:

In [ ]:
# AWS Configuration - UPDATE THESE VALUES
BUCKET_NAME = "energy-grid-YOUR-ID"  # Replace with your bucket name
DYNAMODB_TABLE = "GridAnalysis"
LAMBDA_FUNCTION = "optimize-energy-grid"
AWS_REGION = "us-east-1"

# Initialize AWS clients
s3_client = boto3.client("s3", region_name=AWS_REGION)
dynamodb = boto3.resource("dynamodb", region_name=AWS_REGION)
lambda_client = boto3.client("lambda", region_name=AWS_REGION)

# Verify bucket access
try:
    s3_client.head_bucket(Bucket=BUCKET_NAME)
    print(f"✓ S3 bucket accessible: {BUCKET_NAME}")
except Exception as e:
    print(f"✗ Cannot access bucket: {e}")
    print("Please update BUCKET_NAME with your actual bucket name")

## Step 1: Generate Sample Grid Data

Create synthetic smart grid data with realistic load profiles, renewable generation, and power quality metrics.

In [ ]:
def generate_grid_data(days=7, interval_minutes=15, location="substation_001"):
    """
    Generate synthetic smart grid time-series data.

    Returns:
        DataFrame with timestamp, location, load, generation, voltage, frequency, renewables
    """
    print(f"Generating {days} days of grid data for {location}...")

    # Time range
    start_time = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(
        days=days
    )
    intervals = days * 24 * (60 // interval_minutes)
    timestamps = [start_time + timedelta(minutes=i * interval_minutes) for i in range(intervals)]

    data = []
    np.random.seed(hash(location) % 2**32)

    for ts in timestamps:
        hour = ts.hour
        day_of_week = ts.weekday()

        # Base load with daily pattern (lower at night, higher during day)
        base_load = 100 + 40 * np.sin(2 * np.pi * (hour - 6) / 24)

        # Weekday vs weekend adjustment
        if day_of_week >= 5:  # Weekend
            base_load *= 0.85

        # Add randomness
        load_mw = max(50, base_load + np.random.normal(0, 8))

        # Generation slightly exceeds load for grid stability
        generation_mw = load_mw * 1.05 + np.random.normal(0, 3)

        # Voltage (nominal 13.8 kV distribution, ±5%)
        voltage_kv = 13.8 + np.random.normal(0, 0.35)

        # Frequency (nominal 60 Hz, ±0.05 Hz for stability)
        frequency_hz = 60.0 + np.random.normal(0, 0.025)

        # Solar generation (peak at noon, zero at night)
        if 6 <= hour <= 18:
            solar_factor = np.sin(np.pi * (hour - 6) / 12)
            solar_mw = 25 * solar_factor + np.random.normal(0, 3)
            solar_mw = max(0, solar_mw)
        else:
            solar_mw = 0

        # Wind generation (variable, less predictable)
        wind_base = 18 + 12 * np.sin(2 * np.pi * hour / 24)
        wind_mw = max(0, wind_base + np.random.normal(0, 6))

        # Power factor (0.85-0.98)
        power_factor = np.clip(0.92 + np.random.uniform(-0.05, 0.06), 0.85, 0.98)

        data.append(
            {
                "timestamp": ts.isoformat(),
                "location": location,
                "load_mw": round(load_mw, 2),
                "generation_mw": round(generation_mw, 2),
                "voltage_kv": round(voltage_kv, 3),
                "frequency_hz": round(frequency_hz, 4),
                "solar_mw": round(solar_mw, 2),
                "wind_mw": round(wind_mw, 2),
                "power_factor": round(power_factor, 3),
            }
        )

    df = pd.DataFrame(data)
    print(f"✓ Generated {len(df)} data points")
    return df


# Generate data for 3 substations
locations = ["substation_001", "substation_002", "substation_003"]
grid_data_all = {}

for location in locations:
    grid_data_all[location] = generate_grid_data(days=7, location=location)

print(f"\n✓ Generated data for {len(locations)} substations")

## Step 2: Visualize Generated Data

Preview the synthetic grid data before uploading to AWS.

In [ ]:
# Display sample data
sample_location = "substation_001"
df_sample = grid_data_all[sample_location]

print(f"Sample data from {sample_location}:")
print(df_sample.head(10))

print("\nData Statistics:")
print(df_sample.describe().round(2))

# Quick visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

df_sample["hour"] = pd.to_datetime(df_sample["timestamp"]).dt.hour

# Load pattern
axes[0, 0].plot(range(len(df_sample[:96])), df_sample["load_mw"][:96], "b-", linewidth=2)
axes[0, 0].set_title("Load Profile (24 hours)", fontsize=12, fontweight="bold")
axes[0, 0].set_xlabel("Time Interval")
axes[0, 0].set_ylabel("Load (MW)")
axes[0, 0].grid(True, alpha=0.3)

# Renewable generation
axes[0, 1].plot(
    range(len(df_sample[:96])), df_sample["solar_mw"][:96], "orange", label="Solar", linewidth=2
)
axes[0, 1].plot(
    range(len(df_sample[:96])), df_sample["wind_mw"][:96], "green", label="Wind", linewidth=2
)
axes[0, 1].set_title("Renewable Generation (24 hours)", fontsize=12, fontweight="bold")
axes[0, 1].set_xlabel("Time Interval")
axes[0, 1].set_ylabel("Generation (MW)")
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Voltage distribution
axes[1, 0].hist(df_sample["voltage_kv"], bins=30, color="purple", alpha=0.7, edgecolor="black")
axes[1, 0].axvline(13.8, color="red", linestyle="--", linewidth=2, label="Nominal (13.8 kV)")
axes[1, 0].set_title("Voltage Distribution", fontsize=12, fontweight="bold")
axes[1, 0].set_xlabel("Voltage (kV)")
axes[1, 0].set_ylabel("Frequency")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Frequency distribution
axes[1, 1].hist(df_sample["frequency_hz"], bins=30, color="teal", alpha=0.7, edgecolor="black")
axes[1, 1].axvline(60.0, color="red", linestyle="--", linewidth=2, label="Nominal (60 Hz)")
axes[1, 1].set_title("Frequency Distribution", fontsize=12, fontweight="bold")
axes[1, 1].set_xlabel("Frequency (Hz)")
axes[1, 1].set_ylabel("Frequency")
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Data visualization complete")

## Step 3: Upload Data to S3

Upload generated grid data to S3, which will trigger Lambda processing.

In [ ]:
# Create local directory for data
data_dir = Path("../sample_data")
data_dir.mkdir(parents=True, exist_ok=True)

uploaded_files = []

for location, df in grid_data_all.items():
    # Save to CSV
    filename = f"grid_data_{location}_{datetime.now().strftime('%Y%m%d')}.csv"
    filepath = data_dir / filename
    df.to_csv(filepath, index=False)

    # Upload to S3
    s3_key = f"raw/{filename}"

    try:
        s3_client.upload_file(str(filepath), BUCKET_NAME, s3_key)
        print(f"✓ Uploaded: s3://{BUCKET_NAME}/{s3_key}")
        uploaded_files.append(s3_key)
    except Exception as e:
        print(f"✗ Upload failed for {filename}: {e}")

print(f"\n✓ Uploaded {len(uploaded_files)} files to S3")
print("\n⏱ Lambda will process files automatically (if S3 trigger configured)")
print("   Or manually invoke Lambda in next step")

## Step 4: Invoke Lambda Processing (Optional)

Manually trigger Lambda to process uploaded files if auto-trigger is not configured.

In [ ]:
# Manually invoke Lambda for each uploaded file
for s3_key in uploaded_files:
    # Create S3 event payload
    payload = {"Records": [{"s3": {"bucket": {"name": BUCKET_NAME}, "object": {"key": s3_key}}}]}

    try:
        response = lambda_client.invoke(
            FunctionName=LAMBDA_FUNCTION,
            InvocationType="RequestResponse",
            Payload=json.dumps(payload),
        )

        result = json.loads(response["Payload"].read())
        print(f"✓ Lambda processed: {s3_key}")
        print(f"  Status: {result.get('statusCode')}")

        if result.get("statusCode") == 200:
            body = json.loads(result.get("body", "{}"))
            print(f"  Records: {body.get('records_processed', 0)}")
        else:
            print(f"  Error: {result.get('body')}")

    except Exception as e:
        print(f"✗ Lambda invocation failed for {s3_key}: {e}")

print("\n✓ Lambda processing complete")
print("⏱ Wait 30 seconds for DynamoDB writes to complete...")

import time

time.sleep(30)

## Step 5: Query Results from DynamoDB

Retrieve processed grid analysis results from DynamoDB.

In [ ]:
def decimal_to_float(obj):
    """Convert Decimal to float for JSON serialization."""
    if isinstance(obj, list):
        return [decimal_to_float(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: decimal_to_float(value) for key, value in obj.items()}
    elif isinstance(obj, Decimal):
        return float(obj)
    return obj


# Query DynamoDB for all locations
table = dynamodb.Table(DYNAMODB_TABLE)
results = []

for location in locations:
    try:
        response = table.query(
            KeyConditionExpression="location = :loc",
            ExpressionAttributeValues={":loc": location},
            ScanIndexForward=False,  # Most recent first
            Limit=50,
        )

        items = response.get("Items", [])
        results.extend(items)
        print(f"✓ Retrieved {len(items)} records for {location}")

    except Exception as e:
        print(f"✗ Query failed for {location}: {e}")

print(f"\n✓ Total records retrieved: {len(results)}")

# Convert to DataFrame
if results:
    # Convert Decimals to floats
    results_clean = decimal_to_float(results)

    # Flatten nested structure
    df_results = []
    for r in results_clean:
        flat = {
            "location": r.get("location"),
            "timestamp": r.get("timestamp"),
            "alert_status": r.get("alert_status"),
            "load_avg_mw": r.get("load_metrics", {}).get("avg_mw"),
            "load_peak_mw": r.get("load_metrics", {}).get("max_mw"),
            "renewable_penetration": r.get("renewable_metrics", {}).get("renewable_penetration"),
            "renewable_pct": r.get("renewable_metrics", {}).get("renewable_percentage"),
            "voltage_avg_kv": r.get("voltage_metrics", {}).get("avg_kv"),
            "frequency_avg_hz": r.get("frequency_metrics", {}).get("avg_hz"),
            "stability_score": r.get("power_quality", {}).get("stability_score"),
            "efficiency_score": r.get("power_quality", {}).get("efficiency_score"),
        }
        df_results.append(flat)

    df_results = pd.DataFrame(df_results)
    print("\nResults DataFrame:")
    print(df_results.head())
else:
    print("⚠ No results found. Check Lambda logs and DynamoDB table.")

## Step 6: Grid Performance Analysis

Analyze load patterns, renewable penetration, and grid stability.

In [ ]:
if len(df_results) > 0:
    print("Grid Performance Summary")
    print("=" * 80)

    # Overall statistics
    print(f"\nSubstations Analyzed: {df_results['location'].nunique()}")
    print(f"Total Observations: {len(df_results)}")

    print("\nLoad Statistics:")
    print(f"  Average Load: {df_results['load_avg_mw'].mean():.2f} MW")
    print(f"  Peak Load: {df_results['load_peak_mw'].max():.2f} MW")

    print("\nRenewable Integration:")
    print(f"  Average Penetration: {df_results['renewable_pct'].mean():.2f}%")
    print(f"  Max Penetration: {df_results['renewable_pct'].max():.2f}%")

    print("\nGrid Stability:")
    print(f"  Average Voltage: {df_results['voltage_avg_kv'].mean():.3f} kV")
    print(f"  Average Frequency: {df_results['frequency_avg_hz'].mean():.4f} Hz")
    print(f"  Stability Score: {df_results['stability_score'].mean():.3f}")

    print("\nAlert Status:")
    print(df_results["alert_status"].value_counts())

    print("=" * 80)
else:
    print("No data available for analysis")

## Step 7: Visualization - Load and Renewable Analysis

In [ ]:
if len(df_results) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))

    # 1. Load by Location
    df_results.groupby("location")["load_avg_mw"].mean().plot(
        kind="bar", ax=axes[0, 0], color="steelblue", alpha=0.8
    )
    axes[0, 0].set_title("Average Load by Substation", fontsize=12, fontweight="bold")
    axes[0, 0].set_ylabel("Load (MW)")
    axes[0, 0].set_xlabel("Location")
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Renewable Penetration
    df_results.groupby("location")["renewable_pct"].mean().plot(
        kind="bar", ax=axes[0, 1], color="green", alpha=0.8
    )
    axes[0, 1].set_title("Renewable Penetration by Substation", fontsize=12, fontweight="bold")
    axes[0, 1].set_ylabel("Renewable %")
    axes[0, 1].set_xlabel("Location")
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Stability Score Distribution
    axes[1, 0].hist(
        df_results["stability_score"], bins=20, color="purple", alpha=0.7, edgecolor="black"
    )
    axes[1, 0].axvline(
        df_results["stability_score"].mean(),
        color="red",
        linestyle="--",
        linewidth=2,
        label=f"Mean: {df_results['stability_score'].mean():.3f}",
    )
    axes[1, 0].set_title("Grid Stability Score Distribution", fontsize=12, fontweight="bold")
    axes[1, 0].set_xlabel("Stability Score")
    axes[1, 0].set_ylabel("Frequency")
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Alert Status Pie Chart
    alert_counts = df_results["alert_status"].value_counts()
    colors = {"normal": "green", "warning": "orange", "critical": "red"}
    pie_colors = [colors.get(label, "gray") for label in alert_counts.index]
    axes[1, 1].pie(
        alert_counts.values,
        labels=alert_counts.index,
        autopct="%1.1f%%",
        colors=pie_colors,
        startangle=90,
    )
    axes[1, 1].set_title("Alert Status Distribution", fontsize=12, fontweight="bold")

    plt.tight_layout()
    plt.show()

    print("✓ Visualization complete")
else:
    print("No data available for visualization")

## Step 8: Power Quality Analysis

In [ ]:
if len(df_results) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Voltage stability
    axes[0].scatter(
        df_results["voltage_avg_kv"],
        df_results["stability_score"],
        c=df_results["load_avg_mw"],
        cmap="viridis",
        s=100,
        alpha=0.6,
    )
    axes[0].axvline(13.8, color="red", linestyle="--", linewidth=2, label="Nominal (13.8 kV)")
    axes[0].set_title("Voltage vs Stability Score", fontsize=12, fontweight="bold")
    axes[0].set_xlabel("Voltage (kV)")
    axes[0].set_ylabel("Stability Score")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    cbar = plt.colorbar(axes[0].collections[0], ax=axes[0])
    cbar.set_label("Load (MW)")

    # Frequency stability
    axes[1].scatter(
        df_results["frequency_avg_hz"],
        df_results["stability_score"],
        c=df_results["renewable_pct"],
        cmap="RdYlGn",
        s=100,
        alpha=0.6,
    )
    axes[1].axvline(60.0, color="red", linestyle="--", linewidth=2, label="Nominal (60 Hz)")
    axes[1].set_title("Frequency vs Stability Score", fontsize=12, fontweight="bold")
    axes[1].set_xlabel("Frequency (Hz)")
    axes[1].set_ylabel("Stability Score")
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
    cbar.set_label("Renewable %")

    plt.tight_layout()
    plt.show()

    print("✓ Power quality analysis complete")
else:
    print("No data available for power quality analysis")

## Step 9: Substation Ranking

Rank substations by overall performance.

In [ ]:
if len(df_results) > 0:
    # Calculate substation performance metrics
    ranking = (
        df_results.groupby("location")
        .agg(
            {
                "load_avg_mw": "mean",
                "load_peak_mw": "max",
                "renewable_pct": "mean",
                "stability_score": "mean",
                "efficiency_score": "mean",
                "alert_status": lambda x: (x == "normal").sum() / len(x) * 100,
            }
        )
        .round(2)
    )

    ranking.columns = [
        "Avg Load (MW)",
        "Peak Load (MW)",
        "Renewable %",
        "Stability",
        "Efficiency",
        "Normal %",
    ]

    # Sort by stability score
    ranking = ranking.sort_values("Stability", ascending=False)

    print("\nSubstation Performance Ranking")
    print("=" * 90)
    print(ranking)
    print("=" * 90)

    # Recommendations
    print("\nRecommendations:")
    print("=" * 90)

    for idx, (location, row) in enumerate(ranking.iterrows(), 1):
        stability = row["Stability"]
        renewable = row["Renewable %"]

        print(f"\n{idx}. {location}")
        print(f"   Stability Score: {stability:.3f}")
        print(f"   Renewable Integration: {renewable:.2f}%")

        if stability > 0.90:
            print("   Status: ✓ EXCELLENT - Grid operating optimally")
        elif stability > 0.85:
            print("   Status: ✓ GOOD - Monitor voltage/frequency variations")
        else:
            print("   Status: ⚠ NEEDS ATTENTION - Review power quality")

        if renewable < 15:
            print("   Opportunity: Increase renewable integration")
        elif renewable > 30:
            print("   Achievement: High renewable penetration")

    print("=" * 90)
else:
    print("No data available for ranking")

## Step 10: Export Results and Generate Report

In [ ]:
if len(df_results) > 0:
    # Create results directory
    results_dir = Path("../results")
    results_dir.mkdir(parents=True, exist_ok=True)

    # Export detailed results
    df_results.to_csv(results_dir / "grid_analysis_results.csv", index=False)
    print("✓ Detailed results exported to: grid_analysis_results.csv")

    # Export ranking
    ranking.to_csv(results_dir / "substation_ranking.csv")
    print("✓ Substation ranking exported to: substation_ranking.csv")

    # Generate summary report
    report = f"""
SMART GRID OPTIMIZATION REPORT
Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
{"=" * 80}

EXECUTIVE SUMMARY
Substations Monitored: {df_results["location"].nunique()}
Total Observations: {len(df_results)}

LOAD STATISTICS
Average Load: {df_results["load_avg_mw"].mean():.2f} MW
Peak Load: {df_results["load_peak_mw"].max():.2f} MW
Min Load: {df_results["load_avg_mw"].min():.2f} MW

RENEWABLE INTEGRATION
Average Penetration: {df_results["renewable_pct"].mean():.2f}%
Maximum Penetration: {df_results["renewable_pct"].max():.2f}%

GRID STABILITY
Average Voltage: {df_results["voltage_avg_kv"].mean():.3f} kV (Nominal: 13.8 kV)
Average Frequency: {df_results["frequency_avg_hz"].mean():.4f} Hz (Nominal: 60.0 Hz)
Stability Score: {df_results["stability_score"].mean():.3f}
Efficiency Score: {df_results["efficiency_score"].mean():.3f}

ALERT STATUS SUMMARY
Normal: {(df_results["alert_status"] == "normal").sum()} ({(df_results["alert_status"] == "normal").sum() / len(df_results) * 100:.1f}%)
Warning: {(df_results["alert_status"] == "warning").sum()} ({(df_results["alert_status"] == "warning").sum() / len(df_results) * 100:.1f}%)
Critical: {(df_results["alert_status"] == "critical").sum()} ({(df_results["alert_status"] == "critical").sum() / len(df_results) * 100:.1f}%)

TOP PERFORMING SUBSTATIONS
"""

    for idx, (location, row) in enumerate(ranking.head(3).iterrows(), 1):
        report += f"{idx}. {location}: Stability {row['Stability']:.3f}, Renewable {row['Renewable %']:.2f}%\n"

    report += f"""
KEY FINDINGS
- Grid stability is {"excellent" if df_results["stability_score"].mean() > 0.90 else "good" if df_results["stability_score"].mean() > 0.85 else "needs improvement"}
- Renewable integration at {df_results["renewable_pct"].mean():.1f}% average
- {"No critical" if (df_results["alert_status"] == "critical").sum() == 0 else "Some"} alerts detected

RECOMMENDATIONS
1. Continue monitoring voltage and frequency stability
2. Optimize renewable integration for substations below 20%
3. Investigate and resolve any warning/critical alerts
4. Consider demand response programs during peak load periods
{"=" * 80}
"""

    # Save report
    with open(results_dir / "grid_optimization_report.txt", "w") as f:
        f.write(report)

    print(report)
    print("\n✓ Report saved to: grid_optimization_report.txt")
    print(f"✓ All results exported to: {results_dir}")
else:
    print("No data available for export")

## Summary

You've completed the Tier 2 Energy Systems - Grid Optimization project!

### What You Learned
- **S3:** Upload and store grid sensor data in AWS
- **Lambda:** Process time-series data with serverless functions
- **DynamoDB:** Store and query grid metrics in NoSQL database
- **SNS:** Set up real-time alerts for grid anomalies
- **Grid Analytics:** Analyze load patterns, renewable integration, stability

### Key Insights
- Load patterns show daily and weekly variations
- Renewable penetration varies by time of day (solar) and location (wind)
- Grid stability correlates with voltage and frequency stability
- Peak demand identification enables optimization strategies

### Next Steps
1. **Advanced Analysis:** Add load forecasting, anomaly detection, optimization algorithms
2. **Scale Up:** Process multiple regions, 1000+ substations
3. **Real-time:** Connect to live SCADA/EMS systems
4. **ML Integration:** Train models for demand prediction, fault detection
5. **Tier 3:** Automate with CloudFormation infrastructure as code

### Don't Forget!
**Delete AWS resources** when done to avoid charges:
```bash
# See cleanup_guide.md for detailed instructions
aws s3 rm s3://BUCKET_NAME --recursive
aws s3 rb s3://BUCKET_NAME
aws dynamodb delete-table --table-name GridAnalysis
aws lambda delete-function --function-name optimize-energy-grid
```

---

**Congratulations on completing this AWS energy systems project!**